In [1]:
from stelar.client import Client, TaskSpec
c = Client(base_url="https://klms.stelar.gr", username="dpetrou", password="1234")

In [2]:
# model checkpoints
model_ensemble = c.datasets['3d-unet-checkpoints-for-crop-classification-ucb-pilot-b']

# 30tyq labels for crops
labels_30tyq = c.datasets['eurocrops-austria-2021-geometry-only'].resources[1]

# actual LAI in .npy format
actual_lai = c.datasets['30tyq-lai-npy-measures']


## Create the Task specification

In [3]:
t = TaskSpec(name="Crop Classification Experiment")

# Set up the inputs
t.i(actual_LAI=actual_lai.id)

t.i(spatial_labels=labels_30tyq)

t.i(models_in_ensemble=model_ensemble.id)

# Set a local alias for a dataset to store the output to
t.d(alias="d0", dset=actual_lai)

# Set the output destination
t.o(predictions={
    "url": "s3://vista-bucket/Pilot_B/UCB/experiments/crop-cl-experiment/",
    "resource": {
        "name": "Predictions full set directory",
        "relation": "predictions",
    },
    "dataset": "d0"
})

# Set the parameters 


t.p(months_chosen=["Feb_Aug", "May_Aug", "Jun_Oct", "Jan_Aug"])
t.p(upper_left_x=704855.0)
t.p(upper_left_y=4995145.0)
t.p(pixel_size=10.0)
t.p(crs_code="EPSG:32630")

t.p(Feb_Aug_crop_group=[33, 36, 41, 34, 37, 40])
t.p(Feb_Aug_subgroup1=[34, 37, 40])
t.p(Feb_Aug_subgroup2=[33, 36, 41])

t.p(May_Aug_crop_group=[2, 15, 20, 21, 23, 28])
t.p(May_Aug_subgroup1=[2, 25, 20])
t.p(May_Aug_subgroup2=[21, 23, 28])

t.p(Jun_Oct_crop_group=[8, 30, 9, 18, 19, 7])
t.p(Jun_Oct_subgroup1=[8, 9, 30])
t.p(Jun_Oct_subgroup2=[7, 18, 19])

t.p(Jan_Aug_crop_group=[4, 7, 9])
t.p(Jan_Aug_subgroup1=[4, 7, 9])
t.p(num_eval_patches=300)


In [4]:
proc = c.processes['dda5b7cd-62a8-4b68-b4a9-d68c002b6b69']

crop_classification_t = proc.run(t)

print(f"Monitor the task here: https://klms.stelar.gr/stelar/console/v1/task/{proc.id}/{crop_classification_t.id}")

Monitor the task here: https://klms.stelar.gr/stelar/console/v1/task/dda5b7cd-62a8-4b68-b4a9-d68c002b6b69/6c46908f-1596-4ea0-afb0-1f2b08ac5638


In [ ]:
import subprocess

image_name = "img.stelar.gr/stelar/crop-classification:latest"

ts_id = str(crop_classification_t.id)
ts_signature = crop_classification_t.signature

docker_command = [
    "docker",
    "run",
    #"--rm",
    #"--gpus",
    #"all",
    image_name,
    "Bearer " + c.token,
    c.api_url.rstrip("/api/"),
    ts_id,
    ts_signature,
]

try:
    subprocess.run(docker_command, check=True)
    print("Docker command executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error occurred while executing Docker command: {e}")

Unable to find image 'img.stelar.gr/stelar/crop-classification:latest' locally
latest: Pulling from stelar/crop-classification
09208cb16939: Pulling fs layer
a9ffe18d7fdb: Pulling fs layer
e73850a50582: Pulling fs layer
19fb8589da02: Pulling fs layer
eb5ac1b0a309: Pulling fs layer
51725a7a2af3: Pulling fs layer
207e52cc243f: Pulling fs layer
19fb8589da02: Waiting
bc1bf1eefac0: Pulling fs layer
6949f85d6963: Pulling fs layer
fb927dfdab75: Pulling fs layer
7532514b5d66: Pulling fs layer
eb5ac1b0a309: Waiting
51725a7a2af3: Waiting
1499a5cf2b73: Pulling fs layer
207e52cc243f: Waiting
bc1bf1eefac0: Waiting
afd428207ace: Pulling fs layer
6949f85d6963: Waiting
2683d3a2a492: Pulling fs layer
fb927dfdab75: Waiting
be7acacefdcd: Pulling fs layer
9471241ec0e9: Pulling fs layer
629bbc3550f0: Pulling fs layer
1499a5cf2b73: Waiting
afd428207ace: Waiting
2683d3a2a492: Waiting
be7acacefdcd: Waiting
9471241ec0e9: Waiting
629bbc3550f0: Waiting
7532514b5d66: Waiting
a9ffe18d7fdb: Verifying Checksum
a9ffe

[STELAR INFO] Performing cURL to fetch tool input...
[STELAR INFO] Input fetched and written to input.json
params {'Feb_Aug_crop_group': [33, 36, 41, 34, 37, 40], 'Feb_Aug_subgroup1': [34, 37, 40], 'Feb_Aug_subgroup2': [33, 36, 41], 'Jan_Aug_crop_group': [4, 7, 9], 'Jan_Aug_subgroup1': [4, 7, 9], 'Jun_Oct_crop_group': [8, 30, 9, 18, 19, 7], 'Jun_Oct_subgroup1': [8, 9, 30], 'Jun_Oct_subgroup2': [7, 18, 19], 'May_Aug_crop_group': [2, 15, 20, 21, 23, 28], 'May_Aug_subgroup1': [2, 25, 20], 'May_Aug_subgroup2': [21, 23, 28], 'crs_code': 'EPSG:32630', 'months_chosen': ['Feb_Aug', 'May_Aug', 'Jun_Oct', 'Jan_Aug'], 'num_eval_patches': 300, 'pixel_size': 10.0, 'upper_left_x': 704855.0, 'upper_left_y': 4995145.0}
which season here ['Feb_Aug', 'May_Aug', 'Jun_Oct', 'Jan_Aug']
done saving LAI
done saving labels
trained_models[ts] 3D_unet_g4_h4_crop_21_23_28_epoch_f.h5
trained_models[ts] 3D_unet_g4_h4_crop_2_15_20_epoch_f.h5
trained_models[ts] 3D_unet_g4_h4_crop_34_37_40_epoch_f.h5
trained_models[t

2025-11-03 15:55:31.532416: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 15:55:31.581871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-03 15:55:32.333814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.1
2.13.1
Segmentation Models: using `tf.keras` framework.
Found GPU at: 
2.13.1
2.13.1
Found GPU at: 
2.13.1
2.13.1
Found GPU at: 
starting to evaluate
filepaths ['./dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_00.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_01.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_02.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_03.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_04.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_05.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_06.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_07.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_08.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_09.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_10.tif', './dataset/france2/processed_lai_npy2/30TYQ_LAI_2022_measure_1

1/1 [==============================] - 1s 538ms/step


1/1 [==============================] - 1s 536ms/step
